In [2]:
%matplotlib inline
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception  # TensorFlow ONLY
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [3]:
models_filename = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/v8-vgg16-model-1/v8_vgg16_model_1.h5'
image_dir = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/food-101/images'
image_size = (224, 224)
batch_size = 16
epochs = 80

In [4]:
# 5gb of images won't fit in my memory. use datagenerator to go across all images.
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
zoom_range = 0,
width_shift_range = 0,
height_shift_range=0,
rotation_range=0)

train_generator = train_datagen.flow_from_directory(
image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

num_of_classes = len(train_generator.class_indices)

Found 101000 images belonging to 101 classes.


In [5]:
model = VGG16(weights=None, include_top=False, input_shape=(image_size[0], image_size[1], 3))

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(101*2, activation="relu")(x)
x = Dense(101*2, activation="relu")(x)
predictions = Dense(101, activation="softmax")(x)
model_final = Model(input=model.input, output=predictions)
model_final.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model_final.load_weights(models_filename)

2024-07-08 17:01:37.194666: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-07-08 17:01:37.194705: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-07-08 17:01:37.194719: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-07-08 17:01:37.194945: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-08 17:01:37.194965: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
preds = model_final.evaluate_generator(train_generator, steps=800, workers=8, use_multiprocessing=True)
preds

In [ ]:
# routine for human evaluation - use the generator so we can see how well it can predict
for n in range(100):
    _ = train_generator.next()
    image, classifier = (_[0][0],_[1][0]) # take the first image from the batch
    index = np.argmax(classifier)
    answer = list(train_generator.class_indices.keys())[index]
    predicted = model_final.predict(np.asarray([image]))
    predicted_answer_index = np.argmax(predicted[0])
    predicted_answer = list(train_generator.class_indices.keys())[predicted_answer_index]

    plt.imshow(image)
    plt.show()

    print('correct answer is: ', answer)
    print()
    print('CNN thinks it''s:', predicted_answer)